In [1]:
from headParsing import find_head

In [2]:
import stanza
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency')


2022-04-30 18:20:06 INFO: Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| constituency | wsj      |

2022-04-30 18:20:06 INFO: Use device: gpu
2022-04-30 18:20:06 INFO: Loading: tokenize
2022-04-30 18:20:10 INFO: Loading: pos
2022-04-30 18:20:10 INFO: Loading: constituency
2022-04-30 18:20:11 INFO: Done loading processors!


In [7]:
tests= ['Films shot in New Jersey', 'American comedy films', '1978 debut albums', 'Allegories of humility',
        '1983 in water transport in Japan', 'Civil parishes in North Yorkshire', 'The New Orleans Bee October 1908',
        'Socata TB-10 at Flugplatz Uetersen', 'Portrait paintings by Alexei Harlamov', 'Family portraits of Russia',
        'Maria Pavlovna Abamelik-Lazareva (Demidova)',  'Historical photographs of Kano', 'Maps of Vigneux-sur-Seine', 
        'California Digital Library', 'Old books from American Libraries', 'Schlosspark Vösendorf', 'Comedy films of the United States']


for test in tests:
    print(test, '\t- head: ' + find_head(test, nlp))

Films shot in New Jersey 	- head: Films
American comedy films 	- head: comedy films
1978 debut albums 	- head: debut albums
Allegories of humility 	- head: Allegories
1983 in water transport in Japan 	- head: 1983
Civil parishes in North Yorkshire 	- head: parishes
The New Orleans Bee October 1908 	- head: New Orleans Bee October
Socata TB-10 at Flugplatz Uetersen 	- head: Socata TB
Portrait paintings by Alexei Harlamov 	- head: Portrait paintings
Family portraits of Russia 	- head: Family portraits
Maria Pavlovna Abamelik-Lazareva (Demidova) 	- head: Lazareva
Historical photographs of Kano 	- head: photographs
Maps of Vigneux-sur-Seine 	- head: Maps
California Digital Library 	- head: Library
Old books from American Libraries 	- head: books
Schlosspark Vösendorf 	- head: Schlosspark Vösendorf
Comedy films of the United States 	- head: Comedy films


In [8]:
nlp('Comedy films of the United States').sentences[0].constituency

(ROOT (NP (NP (NN Comedy) (NNS films)) (PP (IN of) (NP (DT the) (NNP United) (NNPS States)))))